In [1]:
source('cascade_helper.R')


In [2]:
p_thr <- 5e-9


In [3]:
outd <- file.path(getwd(), 'out_v3')


In [4]:
traits <- fread('cascade.input.files.tsv') %>% drop_na(c('imp')) %>% filter(imp != '')


In [5]:
traits %>% dim()

[1] 35  4

In [6]:
imp_anno_df <- traits %>% 
read_imp_sumstats_all(p_thr) %>%
annotate_imp_df()


In [7]:
imp_anno_df %>% count(CHROM) %>% arrange(suppressWarnings(as.numeric(CHROM)), CHROM)

CHROM,n
<chr>,<int>
1,76
2,68
3,61
4,46
5,51
6,44
7,43
8,27
9,41


In [8]:
imp_anno_df %>% count(name)

name,n
<chr>,<int>
Alanine aminotransferase,25
Albumin,39
Alkaline phosphatase,32
Apolipoprotein A,35
Apolipoprotein B,20
Aspartate aminotransferase,28
AST to ALT ratio,34
C-reactive protein,25
Calcium,20


In [9]:
imp_anno_df %>%
fwrite(file.path(outd, 'cascade.imp.hits.auto_and_X.tsv'), sep='\t')


In [10]:
imp_anno_df %>%
filter(CHROM != 'X') %>%
fwrite(file.path(outd, 'cascade.imp.hits.tsv'), sep='\t')


In [11]:
imp_anno_df %>% 
mutate(is_X = (CHROM == 'X')) %>%
select(is_rare, is_outside_of_MHC, is_X, ID) %>%
unique() %>%
count(is_X, is_outside_of_MHC, is_rare)

is_X,is_outside_of_MHC,is_rare,n
<lgl>,<lgl>,<lgl>,<int>
FALSE,FALSE,FALSE,2
FALSE,TRUE,FALSE,708
TRUE,TRUE,FALSE,112


```
gdrive upload -p 1fGlnrj8Zu1Pox3fmxmFKdcbNVWiN9vhd out_v3/cascade.imp.hits.tsv
zstd -15 --rm out_v3/cascade.imp.hits.tsv
```